<a href="https://colab.research.google.com/github/titanmaulana/KerjaKuy/blob/main/Recomenndation_for_Job_Finder_ML_60.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Packages
==

In [ ]:
import numpy as np
import numpy.ma as ma
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.model_selection import train_test_split
import tabulate
#from recsysNN_utils import *
#pd.set_option("display.precision", 1)

In [ ]:
!pip install scikit-surprise


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 772.0/772.0 kB 15.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.3-cp310-cp310-linux_x86_64.whl size=3095479 sha256=cb4ff6136c0e21800b54250dc684120d277f9620f5f4b58b29c1ddfc45a669eb
  Stored in directory: /root/.cache/pip/wheels/a5/ca/a8/4e28def53797fdc4363ca4af740db15a9c2f1595ebc51fb445
Successfully built scikit-surprise


In [ ]:
from surprise import Dataset, Reader, SVD, KNNBasic
from surprise.model_selection import train_test_split

Job Datasets
==

In [ ]:
# Upload file
from google.colab import files
uploaded = files.upload()

Saving data website indeed Jakarta (data 30 May 2023).xlsx to data website indeed Jakarta (data 30 May 2023) (5).xlsx


In [ ]:
import io
# Read Excel File
data = pd.read_excel(io.BytesIO(uploaded.get('data website indeed Jakarta (data 30 May 2023).xlsx')))
xlsx_file = io.BytesIO(uploaded.get('data website indeed Jakarta (data 30 May 2023).xlsx'))

# Read Sheets in excel file
sheet_name = 'clean data'
sheets = pd.read_excel(xlsx_file, sheet_name)

# Import Data to Data Frame
df_job = pd.DataFrame(sheets)
sheets

,No,Posisi,Perusahaan,Lokasi,Salary (minimum per bulan),Tipe Pekerjaan,Kualifikasi,Deskripsi Pekerjaan,Link
0,1,Talent Host Streaming ( Freelance ),THE AESTHETIC SKIN CLINIC,Jakarta,0,Part-time,Wanita\nUsia maksimal 30 tahun\nMemiliki penga...,Melakukan penjualan produk secara live di Tik ...,https://id.indeed.com/company/The-Aesthetic-Sk...
1,2,Data Analyst,Peroptyx,Jakarta,1653356,Part-time,Technical requirements to perform the work\nAc...,Work up to 20 hours per week.\nEarn a competit...,https://id.indeed.com/pagead/clk?mo=r&ad=-6NYl...
2,3,Staff Operational,PT. Supra Boga Lestari,Jakarta,0,Full-time,Pria & Wanita \n18 - 25 tahun \nMin. pengalama...,NaN,https://id.indeed.com/rc/clk?jk=7da78850c5a651...
3,4,Server Tasty Kitchen (Area Jabodetabek),PT. Trans F&B,Jakarta,0,Part-time,Pria atau Wanita\nUsia maksimal 26 tahun\nTing...,NaN,https://id.indeed.com/company/PT.-Trans-F&B/jo...
4,5,Admin Online (Cengkareng),GREEBEL,Jakarta,0,Full-time,Usia Maksimal 30 tahun\nPendidikan Minimal Lul...,NaN,https://id.indeed.com/company/GREEBEL/jobs/Sta...
...,...,...,...,...,...,...,...,...,...
307,308,Internship - Bancassurance Project,Prudential plc,Jakarta,0,Part-time,Final Year Student or Fresh Graduate from ...,Encompassing all aspect of setting up Bancassu...,https://id.indeed.com/rc/clk?jk=f8d772e5724594...
308,309,Barista,Serantau Coffee x Space,Jakarta,1500000,Part-time,Pria/Wanita berumur maksimal 35 tahun pertang...,Menemui pelanggan dan memberi informasi kepada...,https://id.indeed.com/company/Serantau-Coffee/...
309,310,Course Consultant (CC),Kukche Languages,Jakarta,0,Full-time,"Wanita, usia 20-40 tahun. Sehat jasmani dan ro...",Memberikan konsultasi perencanaan pendidikan k...,https://id.indeed.com/rc/clk?jk=18332c615c58ce...
310,311,SPG - PIK (Produk non halal),PT. Bali Highlands Organik,Jakarta,4000000,Part-time,"Wanita, Pendidikan minimal SMA/SMK. Maksimal u...",Mempromosikan dan merekomendasikan produk kepa...,https://id.indeed.com/company/PT.-Bali-Highlan...


# Recommendation Code


Using SVD
==

In [ ]:
# User Data for SVD
user_data = {
    'UserID': ['U001', 'U002', 'U003'],
    'Nama': ['Asep', 'Endah', 'Muklis'],
    'Keahlian': [['Analytical Mindset','Excellent Research Skills'], ['Bertanggung Jawab', 'Sarjana'], ['Berpengalaman', 'Rajin']],
    'Preferensi': [['Data Analyst'], ['Admin','Tutor'], ['Admin','Sales']]
}
df_users = pd.DataFrame(user_data)

In [ ]:
# Merge Data
df_recommendation = df_users[['UserID', 'Nama']].merge(df_job[['No', 'Posisi']], how='cross')
df_recommendation['Rating'] = 0

reader = Reader(rating_scale=(0, 1))
dataset = Dataset.load_from_df(df_recommendation[['UserID', 'No', 'Rating']], reader)

In [ ]:
# Train Model
model = SVD()
#model = KNNBasic()
trainset = dataset.build_full_trainset()
model.fit(trainset)

In [ ]:
# Function to get Job Reccomendation
def get_job_recommendations(user_id, top_n=5):
    job_ids = df_recommendation.loc[df_recommendation['UserID'] == user_id, 'No']
    predictions = []
    for job_id in job_ids:
        prediction = model.predict(user_id, job_id)
        predictions.append((job_id, prediction.est))
    recommendations = sorted(predictions, key=lambda x: x[1], reverse=True)[:top_n]
    recommended_jobs = df_job[df_job['No'].isin([rec[0] for rec in recommendations])]
    return recommended_jobs

In [ ]:
# Prediction Result
user_id = 'U001'
get_job_recommendations(user_id)

,No,Posisi,Perusahaan,Lokasi,Salary (minimum per bulan),Tipe Pekerjaan,Kualifikasi,Deskripsi Pekerjaan,Link
8,9,Host Live Streaming Tik Tok Shop,PT Eramas Agri Internasional,Jakarta,2500000,Part-time,Maksimal usia 25 Tahun\nPendidikan Terakhir SM...,Menjadi host selama berjalannya live streaming...,https://id.indeed.com/company/PT-Eramas-Agri-I...
82,83,Leader Cook Jakarta,D'cost Group,Jakarta,4500000,Full-time,"Kandidat harus memiliki setidaknya SMA, Diplom...",NaN,https://id.indeed.com/company/D'cost-Group/job...
92,93,Cash Reflenish,PT Cakrawala Mitra Bersama,Jakarta,0,Full-time,Pengalaman/ Non Pengalaman \nPendidikan Minima...,NaN,https://id.indeed.com/rc/clk?jk=845ddaddef801c...
176,177,Halal Supervisor (Penyelia Halal),深圳雅各之星实业有限公司,Jakarta,2361512,Part-time,"Fluent in spoken English,\nStrong writing skil...",HAS23000 standard training for food/drug/cosme...,https://id.indeed.com/company/%E6%B7%B1%E5%9C%...
303,304,Barista (Jakarta Barat),Nam Heong Ipoh,Jakarta,0,Full-time,Pria/Wanita\nPendidikan minimal SMA/K\nBerpeng...,Melaksanakan tugas sebagai bartender dan baris...,https://id.indeed.com/rc/clk?jk=3854fec47fd37b...


Using TfidfVectorizer
==

In [ ]:
# Library
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
# Upload file
from google.colab import files
users_data_uploaded = files.upload()

Saving User Data.xlsx to User Data (1).xlsx


In [ ]:
# Proccess Data
jobs_data = sheets
users_data = pd.read_excel(io.BytesIO(users_data_uploaded.get('User Data.xlsx')))
users_data

,UserID,Nama,Keahlian,Preferensi
0,1,Asep,"Analytical Mindset, Excellent Research Skills",Data Analyst
1,2,Amanah,"Bertanggung Jawab, Sarjana",Admin
2,3,Aminah,"Sarjana, Rajin",Tutor


In [ ]:
# Vector Representation for user and job qualification
tfidf_vectorizer = TfidfVectorizer()
job_descriptions = tfidf_vectorizer.fit_transform(jobs_data['Kualifikasi'])
user_profiles = tfidf_vectorizer.transform(users_data['Keahlian'])

In [ ]:
# Compute similarity between user_profiles and job_descriptions using cosine similarity
similarity_scores = cosine_similarity(user_profiles, job_descriptions)

In [ ]:
# Prediction result
user_id = 0 # Based on Index (beside UserID)
top_jobs_indices = np.argsort(similarity_scores[user_id])[::-1]
recommended_jobs = jobs_data.iloc[top_jobs_indices[:5]]  # Make 5 list of Job reccomendation
recommended_jobs # Recommendation Jobs for Asep

,No,Posisi,Perusahaan,Lokasi,Salary (minimum per bulan),Tipe Pekerjaan,Kualifikasi,Deskripsi Pekerjaan,Link
292,293,Apprenticeship – Digital Marketing for App Cam...,Prudential plc,Jakarta,0,Full-time,Final year or fresh graduate from any major\nW...,Creative Development\nOwning creative asset de...,https://id.indeed.com/rc/clk?jk=02ae5c901fabc7...
263,264,User Researcher (Freelance),TIYA PTE. LTD.,Jakarta,0,Part-time,Curiosity about our users.\nExperience in work...,NaN,https://id.indeed.com/company/Tiya-Inc./jobs/U...
1,2,Data Analyst,Peroptyx,Jakarta,1653356,Part-time,Technical requirements to perform the work\nAc...,Work up to 20 hours per week.\nEarn a competit...,https://id.indeed.com/pagead/clk?mo=r&ad=-6NYl...
115,116,Customer Service Specialist (REMOTE - Part Tim...,ELITE LINGUISTIC NETWORK,Jakarta,2500000,"Full-time, Part-time",Adequate English speaking and writing skills t...,Liaise with the customer (the adult learners) ...,https://id.indeed.com/company/Elite-Asia-Group...
103,104,Xccelerate+ Commercial Management Trainee,Colgate-Palmolive,Jakarta,0,Full-time,Degree or Master’s degree holder with CGPA 3.2...,Xccelerate+ Commercial Management Trainee Prog...,https://id.indeed.com/rc/clk?jk=d8440e175d4353...
